In [1]:
!nvidia-smi

Fri Apr 22 12:20:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import os
import keras.metrics
import matplotlib.pyplot as plt
from imutils import paths
from sklearn.model_selection import train_test_split

In [4]:
test_dir =  '/content/drive/MyDrive/Major Project/Datasets/Plant Disease Dataset/plantvillage dataset/color/'

In [5]:
fruits = []
fruits_image = []
for i in os.listdir(test_dir):
    for image_filename in os.listdir(test_dir + i):
        fruits.append(i) # name of the fruit 
        fruits_image.append(i + '/' + image_filename)

In [6]:
test_fruits = pd.DataFrame(fruits, columns=["Fruits"])
test_fruits["Fruits Image"] = fruits_image

In [7]:
test_fruits

,Fruits,Fruits Image
0,Apple___Apple_scab,Apple___Apple_scab/01a66316-0e98-4d3b-a56f-d78...
1,Apple___Apple_scab,Apple___Apple_scab/00075aa8-d81a-4184-8541-b69...
2,Apple___Apple_scab,Apple___Apple_scab/01f3deaa-6143-4b6c-9c22-620...
3,Apple___Apple_scab,Apple___Apple_scab/0a5e9323-dbad-432d-ac58-d29...
4,Apple___Apple_scab,Apple___Apple_scab/2e264ee0-7679-4ad2-a19a-1ac...
...,...,...
54300,Tomato___healthy,Tomato___healthy/4777e41f-56f7-4f8f-9c81-33bd4...
54301,Tomato___healthy,Tomato___healthy/9e43d49b-abbe-4b9b-8566-42586...
54302,Tomato___healthy,Tomato___healthy/bc825946-3ee5-4269-8ba4-202af...
54303,Tomato___healthy,Tomato___healthy/c3dcca18-f12f-4a4d-a90c-563de...


In [8]:
image_paths=[]
data_labels=[]

In [9]:
for i in range(len(test_fruits)):
  path=test_fruits['Fruits Image'][i]
  path=test_dir+path
  label=test_fruits['Fruits'][i]
  image_paths.append(path)
  data_labels.append(label)

In [10]:
# Training and Validation split
from sklearn.model_selection import train_test_split
(tdpl, image_paths, tdcl, data_labels) = train_test_split(image_paths,
                                                                                                        data_labels,
                                                                                                        test_size = 0.3,
                                                                                                        stratify = data_labels,
                                                                                                        random_state = 101)

In [11]:
df_test = pd.DataFrame({"Path":image_paths,"Class":data_labels})

In [12]:
df_test

,Path,Class
0,/content/drive/MyDrive/Major Project/Datasets/...,Grape___Esca_(Black_Measles)
1,/content/drive/MyDrive/Major Project/Datasets/...,Tomato___Tomato_Yellow_Leaf_Curl_Virus
2,/content/drive/MyDrive/Major Project/Datasets/...,Tomato___Late_blight
3,/content/drive/MyDrive/Major Project/Datasets/...,Tomato___Bacterial_spot
4,/content/drive/MyDrive/Major Project/Datasets/...,Squash___Powdery_mildew
...,...,...
16287,/content/drive/MyDrive/Major Project/Datasets/...,Tomato___Bacterial_spot
16288,/content/drive/MyDrive/Major Project/Datasets/...,Orange___Haunglongbing_(Citrus_greening)
16289,/content/drive/MyDrive/Major Project/Datasets/...,Potato___Late_blight
16290,/content/drive/MyDrive/Major Project/Datasets/...,Tomato___Bacterial_spot


In [14]:
from keras.models import load_model

trained_model = load_model("/content/drive/MyDrive/Major Project/Training Files/Dataset 2/Transfer Learning/Ratio 2/Models/vgg16-ep05.h5")

In [15]:
trained_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [17]:
ts_data = ImageDataGenerator()
testing_df = ts_data.flow_from_dataframe(df_test, x_col = "Path",y_col = "Class", batch_size=1, target_size=(128, 128))

Found 16292 validated image filenames belonging to 38 classes.


In [18]:
trained_model.evaluate(testing_df)

  498/16292 [..............................] - ETA: 1:10:31 - loss: 0.7239 - accuracy: 0.8233 - precision: 0.8959 - recall: 0.7430 - auc: 0.9855 - tp: 370.0000 - fp: 43.0000 - tn: 18383.0000 - fn: 128.0000

KeyboardInterrupt: ignored